In [152]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer as wnl
from nltk.stem.snowball import GermanStemmer as gs

In [153]:
contractions = ["it's", "he's","she's","that's", "what's", "there's",\
                        "[newline]", "'m", "'ve","n't", "'ll","'re", "won't", "'d", "'s"]
fixes = ["it is", "he is","she is","that is", "what is", "there is",\
                 " ", " am", " have", " not", " will", " are", "will not", " would", ""]
stop_en = set(stopwords.words('english'))
stop_germ = set(stopwords.words('german'))

In [154]:
de_dic = {}
with open('german.dic', 'r', encoding='latin-1') as f:
    for row in f:
        if len(row) >1:
            de_dic[row.strip().lower()] = 1
en_dic = {}
with open('english.dic', 'r',) as f:
    for row in f:
        en_dic[row.strip().lower()] = 1

In [155]:
raw_text = open('tweets.csv').read()
tab_seperated = [item.split('\t') for item in raw_text.split('\n') if len(item.split('\t')) >= 4]

In [186]:
data = {}
for i in range(len(tab_seperated)):
    data[tab_seperated[i][1]] = tab_seperated[i][4]
    
data_index = data.copy()    
for tweet in data_index.keys():
    data_index[tweet] = data_index[tweet].lower()
    for i in range(len(contractions)):
        if contractions[i] in data_index[tweet]:
            data_index[tweet] = data_index[tweet].replace(contractions[i], fixes[i])
    data_index[tweet] = re.sub('https?[^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub('[@#][^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'[0-9][^\s]+', ' ' , data_index[tweet])
    data_index[tweet] = re.sub(r'[^a-zäöüß\s]', ' ', data_index[tweet])
    data_index[tweet] = re.sub(r'[^\w\s]', ' ' , data_index[tweet])
    
terms = {}
term_index = {}
for num,tweet in data_index.items():
    for word in tweet.split():
        if word in term_index:
            term_index[word].append(num)
        elif word not in term_index:
            term_index[word] = [num]
        if word in terms:
            terms[word] += 1
        else:
            terms[word] = 1

for key in term_index.keys():
    term_index[key] = sorted(term_index[key])

In [187]:
def is_language(term):
        de_char = ['ä','ö','ü','ß']
        de_score = 0
        en_score = 0
        for post in term_index[term]:
            for i in data_index[post].lower().strip().split():
                for char in de_char:
                    if char in i:
                        de_score +=1
                de_score +=1
                if i in stop_germ:
                    de_score += 1
                if i in stop_en:
                    en_score += 1
        if de_score>en_score:
            return 'german'
        elif de_score<en_score:
            return 'english'
        else:
            return None

In [188]:
def top_freq(dict):
        freq = []
        for term, val in dict.items():
            freq.append((val, term))
        freq = sorted(freq)[::-1]
        freq_de = []
        freq_en = []
        for i,j in freq[:200]:
            if is_language(j) == 'german':
                freq_de.append(j)
            else:
                freq_en.append(j)
        return freq_de, freq_en

In [189]:
 def language(post):
        de_char = ['ä','ö','ü','ß']
        de_score = 0
        en_score = 0
        for i in data_index[post].split():
            for char in de_char:
                    if char in i:
                        de_score +=1
            if i in stop_germ or i in top_de:
                de_score += 1
            if i in stop_en or i in top_en:
                en_score += 1
        if de_score>en_score:
            return 'german'
        elif en_score>de_score:
            return 'english'
        else:
            return None

In [210]:
a = [x for x in terms.keys()][:200]

In [218]:
for i in a:
    en_lemma = {}
    tags = ['n','v','a','s','r']
    for tag in tags:
        for lemma in (wnl.lemmatize(wnl,word=i, pos=tag)):
            en_lemma[lemma] = 1 
    for j in en_lemma:
        if (j not in en_dic) and (j not in de_dic):
            print(i)

für
überprüft
erklärbare
können
glückwunsch
eingeschläfert


In [202]:
def get_misspells():
        words =sorted([key for key in terms.keys()])
        de = []
        en = []
        for term in words[:200]:
            g_count = 0
            e_count = 0
            e_posts = []
            g_posts = []
            
            #pos tags for lemmatization generation
            tags = ['n','v','a','s','r']
            en_lemma = {}
            for tag in tags:
                for lemma in (wnl.lemmatize(wnl,word=term, pos=tag)):
                    en_lemma[lemma] = 1 
                    
            #German lemmatiziaton/stemming
            de_lemma = term
            
            for post in term_index[term]:
                if language(post) == 'german':
                    ## get german misspelling count
                    if de_lemma not in de_dic:    
                        g_count += 1
                        g_posts.append(post)
                        
                elif language(post) == 'english':

                    ## get german misspelling count
                    if len([i for i in en_lemma.keys() if i in en_dic]) == 0:
                        e_count += 1
                        e_posts.append(post)
                        
                else:
                    lang = is_language(term)
                    if lang ==  None:
                        continue

                    if lang == 'german':
                        
                        ## get german misspelling count    
                        if de_lemma not in de_dic:

                            g_count += 1
                            g_posts.append(post)
                    else:
                        ## get english misspelling count
                        if len([i for i in en_lemma.keys() if i in en_dic]) == 0:
                            e_count += 1
                            e_posts.append(post)
                            
            if g_count > 0:
                de.append((g_count, term, g_posts))
            if e_count > 0:
                en.append((e_count, term, e_posts))
        return sorted(de)[::-1], sorted(en)[::-1]

In [203]:
de_mis, en_mis = get_misspells()

In [219]:
len(de_mis)

66

In [ ]:
#             term = ''
#             for i in range(len(x)):
#                 try:
#                     if x[i] != [i+2]:
#                         term += x[i]
#                 except:
#                     term += x[i]